# Growth Model

## Initizalizing the Model

In [1]:
using Parameters

@with_kw struct Primitives
    β::Float64 = 0.99 #discount rate
    δ::Float64 = 0.025 #depreciation rate
    α::Float64 = 0.36 #capital share
    k_min::Float64 = 0.01 #capital lower bound
    k_max::Float64 = 75.0 #capital upper bound
    nk::Int64 = 1000 #number of capital grid points
    k_grid::Array{Float64,1} = collect(range(k_min, length = nk, stop = k_max)) #capital grid
end

#structure that holds model results
mutable struct Results
    val_func::Array{Float64, 1} #value function
    pol_func::Array{Float64, 1} #policy function
end

#function for initializing model primitives and results
function Initialize()
    prim = Primitives() #initialize primtiives
    val_func = zeros(prim.nk) #initial value function guess
    pol_func = zeros(prim.nk) #initial policy function guess
    res = Results(val_func, pol_func) #initialize results struct
    prim, res #return deliverables
end

Initialize (generic function with 1 method)

In [2]:
prim, res = Initialize()

(Primitives
  β: Float64 0.99
  δ: Float64 0.025
  α: Float64 0.36
  k_min: Float64 0.01
  k_max: Float64 75.0
  nk: Int64 1000
  k_grid: Array{Float64}((1000,)) [0.01, 0.08506506506506506, 0.16013013013013014, 0.2351951951951952, 0.31026026026026027, 0.3853253253253253, 0.46039039039039037, 0.5354554554554555, 0.6105205205205205, 0.6855855855855856  …  74.32441441441442, 74.39947947947948, 74.47454454454454, 74.5496096096096, 74.62467467467468, 74.69973973973974, 74.7748048048048, 74.84986986986986, 74.92493493493494, 75.0]
, Results([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))

## Solving The Model

The agents solve the following maximization problem:

$$\begin{aligned}\max_{c_t, k_{t+1}}&\quad \sum_{t=0}^{\infty} \ln{(c_t)}\\ s.t.& \quad c_t + k_{t+1} \leq z_t k_t^\alpha + (1-\delta) k_t  \end{aligned}$$

We can solve this problem using using Dynamic Programming:

$$v(k) = \max_{c, k' \in \Gamma(k)} \left\{\log(c) + \beta v(k')\right\}$$

Since the utility function is increasing then we can assume $$c = z k^\alpha + (1-\delta) k  - k'$$

Next we define a fucntion `Bellman` that recieves the parameters of the model and returns the value function:

In [ ]:
function Bellman(prim::Primitives,res::Results)
    @unpack val_func = res #unpack value function
    @unpack k_grid, β, δ, α, nk = prim #unpack model primitives
    v_next = zeros(nk) #next guess of value function to fill
    choice_lower = 1 #for exploiting monotonicity of policy function
    
    for k_index = 1:nk
        k = k_grid[k_index] #value of k
        candidate_max = -Inf #bad candidate max
        budget = k^α + (1-δ)*k #budget

        for kp_index in choice_lower:nk #loop over possible selections of k', exploiting monotonicity of policy function
            c = budget - k_grid[kp_index] #consumption given k' selection
            if c>0 #check for positivity
                val = log(c) + β*val_func[kp_index] #compute value
                if val>candidate_max #check for new max value
                    candidate_max = val #update max value
                    res.pol_func[k_index] = k_grid[kp_index] #update policy function
                    choice_lower = kp_index #update lowest possible choice
                end
            end
        end
    
        v_next[k_index] = candidate_max #update value function
    end
    
    v_next #return next guess of value function
end

# TODO: Explain all this in next markdown cell

Bellman (generic function with 1 method)

**Outter loop**:

* `k`: the current capital is choosen from `k_grid`  a grid of all possible capital levels.
* `budget`: the current budget is $zk^\alpha + (1-\delta)k$, note that in this example $z=1$.
* `candidate_max`: is the initial value of the value function, initialized to something very small (always will be updated).
* `choice_lower`: 

* **Inner loop**:

    * `kp_index`: the index of the next capital level, iterate over all possible capital levels.
    * `c`: the consumption if next period capital is `k_grid[kp_index]`. If consumption is not positive then we inmediatley discard this level.
    * `val`: the value with the selected capital level, if this value is greater than the current value then we update the policy function and update `choice_lower`
    * `v_next`: the value of the value function with the next capital level.

In [5]:
v_next = Bellman(prim, res) 

1000-element Vector{Float64}:
 -1.6591741470563819
 -0.7240986861020424
 -0.41057394607835707
 -0.20675782529969303
 -0.0526822847845916
  0.07241665133489694
  0.17834300810514114
  0.2705533187910241
  0.35241701696170696
  0.42617029767126136
  ⋮
  4.3470154718910035
  4.347984632445973
  4.3489528403643245
  4.3499200975263275
  4.3508864058066745
  4.351851767074507
  4.35281618319343
  4.35377965602154
  4.354742187411443